In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pypsa
import pandas as pd
import os

plt.style.use(["bmh", "../../matplotlibrc"])
sns.set_palette("colorblind")

COUNTRY = "UK"

In [ ]:
files = {
    "VOLL": f"../../results/20240506-70a/network_solved_LT-country+{COUNTRY}-voll+true.nc",
    "PWL-elastic": f"../../results/20240506-70a/network_solved_LT-country+{COUNTRY}-elastic_pwl+default.nc",
}
ns = {k: pypsa.Network(v) for k, v in files.items()}

In [ ]:
def get_annual_revenues(n):
    revenue = (
        n.statistics.revenue(aggregate_time=False)
        .groupby("carrier")
        .sum()
        .T.drop("load", axis=1)
    )
    annual_revenue = revenue.groupby(revenue.index.year).sum()
    to_combine = ["battery charger", "battery discharger"]
    annual_revenue["battery dis-/charging"] = annual_revenue[to_combine].sum(axis=1)
    annual_revenue = annual_revenue.drop(to_combine, axis=1)
    normed_annual_revenue = annual_revenue / annual_revenue.mean() * 1 - 1
    return normed_annual_revenue  # p.u.

In [ ]:
revenues = pd.concat({k: get_annual_revenues(v) for k, v in ns.items()})
revenues.index.names = ["scenario", "year"]
revenues

In [ ]:
revenues_long = pd.melt(
    revenues.reset_index(),
    id_vars=["scenario", "year"],
    var_name="carrier",
    value_name="value",
)

In [ ]:
revenues_long["carrier"] = revenues_long["carrier"].str.replace("fuel cell", "turbine")
revenues_long["carrier"] = revenues_long["carrier"].str.replace(" ", "\n")
revenues_long.sort_values(by="carrier", ascending=False, inplace=True)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 4))

sns.stripplot(
    revenues_long,
    ax=ax,
    orient="v",
    x="carrier",
    y="value",
    hue="scenario",
    dodge=True,
    alpha=0.5,
    size=3,
    jitter=0.2,
    hue_order=["VOLL", "PWL-elastic"],
)
sns.violinplot(
    revenues_long,
    ax=ax,
    orient="v",
    x="carrier",
    y="value",
    hue="scenario",
    dodge=True,
    facecolor="#fff",
    edgecolor="#ccc",
    inner="quart",
    # density_norm="area",
    cut=True,
    hue_order=["VOLL", "PWL-elastic"],
    legend=False,
    bw_adjust=0.5,
)

plt.ylabel("Annual revenue\n[per-unit relative to mean]")
plt.ylim(-10, 20)
plt.yticks(range(-10, 20, 1), minor=True)
plt.axhline(0, color="#444", lw=0.5)
plt.xlabel("")

for ext in ["png", "pdf"]:
    plt.savefig(
        f"../../figures/figure-annual-revenue-{COUNTRY}.{ext}",
        bbox_inches="tight",
        dpi=300,
    )